In [8]:
import requests
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res = requests.get(youbike_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名: str = Field(alias='sna')
    區域: str = Field(alias='sarea')
    時間: str = Field(alias='mday')
    地點: str = Field(alias='ar')
    狀態: bool = Field(alias='act')
    總共數量: int = Field(alias='total')
    可借數量: int = Field(alias='available_rent_bikes')
    可還數量: int = Field(alias='available_return_bikes')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')

    @field_validator('站名')
    @classmethod
    def get_name(cls, value: str) -> str:
        return value.split('_')[-1]

class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root[item]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

import pandas as pd
df1 = pd.DataFrame(all_sites)
df1

下載成功


,站名,區域,時間,地點,狀態,總共數量,可借數量,可還數量,經度,緯度
0,捷運科技大樓站,大安區,2024-06-17 19:12:22,復興南路二段235號前,True,28,4,24,121.54360,25.02605
1,復興南路二段273號前,大安區,2024-06-17 19:11:21,復興南路二段273號西側,True,21,13,8,121.54357,25.02565
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,True,16,2,14,121.54124,25.02429
3,和平公園東側,大安區,2024-06-17 19:05:16,和平東路二段118巷33號,True,11,11,0,121.54282,25.02351
4,辛亥復興路口西北側,大安區,2024-06-17 19:11:21,復興南路二段368號,True,16,11,5,121.54299,25.02153
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:11:21,臺大圖書館西南側,True,30,0,30,121.54031,25.01690
1419,臺大黑森林西側,臺大公館校區,2024-06-17 18:59:20,臺大霖澤館南側,True,20,1,19,121.54347,25.01995
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 18:56:20,臺大獸醫系館南側,True,24,3,21,121.54242,25.01791
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:12:22,臺大體育館東側,True,40,12,28,121.53591,25.02112


In [13]:
#主程式要先跑過，才能在另一組執行
df1[df1['可借數量'] <= 3]

,站名,區域,時間,地點,狀態,總共數量,可借數量,可還數量,經度,緯度
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,True,16,2,14,121.541240,25.024290
13,大安運動中心停車場,大安區,2024-06-17 18:53:21,敦南街76巷28號,True,14,2,12,121.546446,25.020348
16,和平臥龍街口,大安區,2024-06-17 19:11:21,臥龍街1號,True,15,3,12,121.546705,25.024461
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,5,121.533100,25.020930
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),True,50,2,48,121.541420,25.014510
...,...,...,...,...,...,...,...,...,...,...
1415,臺大鄭江樓北側,臺大公館校區,2024-06-17 19:12:16,臺大新聞研究所南側,True,10,1,9,121.543050,25.019000
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:12:16,臺大電機二館南側,True,14,0,14,121.542470,25.018570
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:11:21,臺大圖書館西南側,True,30,0,30,121.540310,25.016900
1419,臺大黑森林西側,臺大公館校區,2024-06-17 18:59:20,臺大霖澤館南側,True,20,1,19,121.543470,25.019950
